# Исследование надежности заемщиков.

**Цель исследования:**

Проверить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

Исследование будет включать следующие шаги:
1. Обзор данных.
2. Предобработка данных.
3. Ответы на вопросы:
* Есть ли зависимость между количеством детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

### Шаг 1. Обзор данных

Составим представление о входных данных. Для этого выведем на экран первые 10 строк таблицы, а также получим общую информацию о ней.

In [1]:
#импорт библиотеки pandas
import pandas as pd
#чтение файла с данными и сохранение в переменную data
data = pd.read_csv('/datasets/data.csv')
#вывод первых 10 строк таблицы на экран
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
#получение информации о данных в таблице
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Таблица состоит из 12 столбцов, согласно документации и полученной информации:
* `children` — количество детей в семье, тип данных — `int`;
* `days_emloyed`— общий трудовой стаж в днях, тип данных — `float`;
* `dob_years` — возраст клиента в годах, тип данных — `int`;
* `education` — уровень образования клиента, тип данных — `object`;
* `education_id` — идентификатор уровня образования, тип данных — `int`;
* `family_status` — семейное положение, тип данных — `object`;
* `family_status_id` — идентификатор семейного положения, тип данных — `int`;
* `gender` — пол клиента, тип данных — `object`;
* `income_type` — тип занятости, тип данных — `object`;
* `debt` — имел ли задолженность по возврату кредитов, тип данных — `int`;
* `total_income` — ежемесячный доход, тип данных — `float`;
* `purpose` — цель получения кредита, тип данных — `object`.

### Шаг 2.1 Заполнение пропусков

В двух столбцах, `days_employed` и `total_income`, количество значений отличается от общего количества, следовательно, в этих столбцах есть пропуски. Столбец `days_employed` будет обработан на следующем шаге, пока заполним пропуски в столбце `total_income`.

In [3]:
#вывод первых 10 строк таблицы с пропущенными значениями в столбце total_income 
data[data['total_income'].isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Можно заметить, что значения в столбцах `days_employed` и `total_income` пропущены в одних и тех же строках, по крайней мере, для первых 10 записей с пропусками. Проверим, справедливо ли это для всей таблицы.

In [4]:
#определяем количество таких записей, для которых в столбце total_income пропущено значение,
#а в столбце days_employed - нет
#data[data['total_income'].isna()].loc[data['days_employed'].isna()==False].shape[0]
data.loc[(data['total_income'].isna()) & (data['days_employed'].isna()==False)].shape[0]

0

Так как нет ни одной строки, для которой было бы справедливо наличие пропуска в столбце `total_income` и отсутствие пропуска в `days_employed`, можно сделать вывод, что пропуски в обоих столбцах находятся в одних и тех же строках. 

Проверим, какую долю составляют пропущенные значения.

In [5]:
#делим количество пустых записей на количество всех записей
(data.shape[0] - data['total_income'].count())/data.shape[0]

0.10099883855981417

Пропущенные значения составляют примерно десятую часть от всех записей в таблице. Скорее всего, пропуски могли возникнуть из-за некорректного заполнения этих сведений, либо же из-за ошибок в системе или при выгрузке данных. Также 10% - это слишком большое количество, чтобы просто удалить пустые записи. Заполнять пустые значения в столбце `total_income` будем медианным значением, так как среднее арифметическое может некорректно характеризовать данные из-за возможных сильно выбивающихся значений дохода.

In [6]:
#замена пустых значений в столбце total_income на медианное значение
data['total_income'] = data['total_income'].fillna(data['total_income'].median())
#проверка на отсутствие пустых значений в столбце после замены
len(data[data['total_income'].isna()])

0

### Шаг 2.2 Проверка данных на аномалии и исправления.

Рассмотрим столбец `days_employed`. В нём присутствуют отрицательные значения, а также значения слишком большие для того, чтобы быть трудовым стажем в днях. Например, для строки с индексом 4 посчитаем трудовой стаж в годах.

In [7]:
#вычисление трудового стажа в годах
data.loc[4, 'days_employed']/365

932.2358138269094

Очевидно, что трудовой стаж не может составлять 932 года. Посмотрим, какая доля в столбце `days_employed` отрицательных значений или значений стажа более 100 лет.

In [8]:
#вычисление доли некорректных или отрицательных значений 
data[(data['days_employed']<0) | (data['days_employed']>100*365)]['days_employed'].count()/data.shape[0]

0.8990011614401858

Почти 90% данных имеют некорректное или отрицательное значение. Такое могло произойти из-за системного сбоя при заполнении таблицы, сбоя при выгрузке данных, либо же значения стажа заполнялись не в днях, а другой единице измерения, например, в часах. 

На предыдущем шаге также было выявлено, что десятая часть данных в столбце `days_employed` пропущена. Заполним ее медианным значением.

In [9]:
#замена пустых значений в столбце days_employed на медианное значение
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())
#проверка на отсутствие пустых значений в столбце после замены
len(data[data['days_employed'].isna()])

0

### Шаг 2.3. Изменение типов данных.

Заменим вещественный тип данных в столбце `total_income` на целочисленный.

In [10]:
#замена в столбце типа float на тип int
data['total_income'] = data['total_income'].astype('int')
#проверка изменения типа данных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

Проверим, есть ли в таблице строки-дубликаты. Для этого применим метод `duplicated()`, а затем посчитаем количество найденных строк-дубликатов.

In [11]:
#находим строки-дубликаты и считаем их количество
data.duplicated().sum()

54

В таблице 54 явных дубликата, они могли возникнуть из-за сбоя системного или при выгрузке данных, либо же данные о каких-то клиентах могли быть случайно занесены несколько раз. Удалим явные дубликаты.

In [12]:
#удаление явных дубликатов и восстановление порядка индексов
data = data.drop_duplicates().reset_index(drop=True)

Теперь проверим наличие неявных дубликатов, посмотрим, из-за каких столбцов строки могут отличаться только из-за регистра букв. Такими столбцами могут быть `education`, `family_status`, `income_type`, `purpose`.

In [13]:
#вывод уникальных значений в столбце education
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [14]:
#вывод уникальных значений в столбце family_status
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [15]:
#вывод уникальных значений в столбце income_type
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [16]:
#вывод уникальных значений в столбце purpose
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Из всех рассмотренных столбцов значения только из-за регистра букв отличаются в столбце `education`. Приведем все значения к нижнему регистру.

In [17]:
#перевод значений в столбце в нижний регистр
data['education'] = data['education'].str.lower()
#вывод уникальных значений для проверки
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Теперь снова проверим, есть ли дубликаты, и посчитаем их количество.

In [18]:
#находим строки-дубликаты и считаем их количество
data.duplicated().sum()

17

После приведения всех данных к одному регистру выявились еще 17 дубликатов, удалим их.

In [19]:
#удаление дубликатов и восстановление порядка индексов
data = data.drop_duplicates().reset_index(drop=True)

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Чтобы оптимизировать работу с данными, уберем из исходной таблицы столбцы `education` и `family_status` и создадим 2 новых датафрейма-"словаря", в которых каждому значению `education` и каждому значению `family_status` будет соответствовать свой идентификатор.

In [20]:
#создание нового датафрейма и удаление дубликатов
education_dict = data[['education_id', 'education']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [21]:
#создание нового датафрейма и удаление дубликатов
family_status_dict = data[['family_status_id', 'family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [22]:
#удаление в исходной таблице столбцов education и family_status
data = data.drop(columns=['education', 'family_status'], axis=1)
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.6. Категоризация дохода.

Создадим в исходной таблице столбец с указанием категории дохода, где:
* 0-30000 - категория `E`;
* 30001-50000 - категория `D`;
* 50001-200000 - категория `C`;
* 200001-1000000 - категория `B`;
* 1000001 и выше - категория `A`. 

Для этого напишем функцию, которая на вход будет получать значение дохода клиента, а возвращать будет категорию.

In [23]:
#функция, возвращающая категорию дохода на основании значения дохода
def total_income_category(total_income):
    if 0 <= total_income <= 30000:
        return 'E'
    elif total_income <= 50000:
        return 'D'
    elif total_income <= 200000:
        return 'C'
    elif total_income <= 1000000:
        return 'B'
    return 'A'

Добавим столбец с категорией дохода с помощью метода `apply()`.

In [24]:
#создание нового столбца с категорией дохода
data['total_income_category'] = data['total_income'].apply(total_income_category)
data.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,-8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,-4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,-5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,-4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,-926.185831,27,0,1,M,компаньон,0,255763,покупка жилья,B
6,0,-2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем,B
7,0,-152.779569,50,1,0,M,сотрудник,0,135823,образование,C
8,2,-6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,-2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C


### Шаг 2.7. Категоризация целей кредита.

Добавим категоризацию целей кредита со следующими категориями:
* операции с автомобилем;
* операции с недвижимостью;
* проведение свадьбы;
* получение образования.

Для начала изучим данные в столбце `purpose` и посмотрим, какие подстроки помогут в категоризации.

In [25]:
#вывод уникальных значений столбца purpose
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

После изучения этих данных напишем функцию, которая на основании подстроки из данных столбца `purpose` будет возвращать категорию целей кредита.

In [26]:
#функция, принимающая на вход цель кредита и возвращающая ее категорию
def purpose_category(purpose):
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'образовани' in purpose:
        return 'получение образования'
    return 'операции с недвижимостью'

Добавим столбец с категорией цели кредита с помощью метода `apply()`.

In [27]:
#создание нового столбца с категорией цели кредита
data['purpose_category'] = data['purpose'].apply(purpose_category)
data.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,-8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,-4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,-5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,-4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,-926.185831,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,-2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,-152.779569,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,-6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,-2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


### Ответы на вопросы.

##### Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?

##### Вывод 1:

Для проверки того, есть ли зависимость между количеством детей и возвратом кредита в срок, создадим сводную таблицу. Так как наличие задолженности по кредиту в таблице обозначается как 1, а отсутствие - 0, то сгруппируем данные по количеству детей и применим функцию нахождения среднего к столбцу `debt`.

In [28]:
#создание сводной таблицы с группировкой по количеству детей
#и со средним значением по столбцу debt
children_pivot = data.pivot_table(index='children', values='debt', aggfunc='mean')
children_pivot

,debt
children,
-1,0.021277
0,0.075438
1,0.092346
2,0.094542
3,0.081818
4,0.097561
5,0.000000
20,0.105263


В полученной таблице есть значение количества детей -1, скорее всего, это означает отсутствие данных о наличии детей у клиента.

Из полученной сводной таблицы видно, что чаще всего не имеют задолженности по кредитам клиенты без детей (примерно 7.5% задолженностей), а также что у многодетных клиентов чаще всего возникают проблемы с возвратом кредита в срок (примерно 10.5% задолженностей). У клиентов с 1-4 детьми примерно одинаковые значения, 8-9.5% задолженностей. 

##### Вопрос 2:

Есть ли зависимость между семейным положением и возвратом кредита в срок?

##### Вывод 2:

Составим сводную таблицу с группировкой по семейному положению, но для начала для удобства объединим таблицу с данными с таблицей-словарем `family_status_dict`.

In [29]:
#объединение таблиц data и family_status_dict
data_final = data.merge(family_status_dict, on='family_status_id', how='left')
#создание сводной таблицы с группировкой по семейному положению
#и со средним значением по столбцу debt
data_final.pivot_table(index='family_status', values='debt', aggfunc='mean')

,debt
family_status,
Не женат / не замужем,0.097509
в разводе,0.071130
вдовец / вдова,0.065693
гражданский брак,0.093471
женат / замужем,0.075452


Из полученной сводной таблицы можно сделать вывод, что вдовцы и вдовы чаще возвращают кредит вовремя (около 6.6% задолженностей), а неженатые/незамужние или клиенты в гражданском браке чаще остальных имеют задолженности по кредиту (около 9.8% и 9.3% соответственно). Женатые или замужние клиенты и клиенты в разводе имеют примерно одинаковый процент задолженностей, около 7-7.5%.

##### Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

##### Вывод 3:

Составим сводную таблицу с группировкой по категории дохода.

In [30]:
#создание сводной таблицы с группировкой по категории дохода
#и со средним значением по столбцу debt
data.pivot_table(index='total_income_category', values='debt', aggfunc='mean')

,debt
total_income_category,
A,0.080000
B,0.070621
C,0.084915
D,0.060000
E,0.090909


Чаще всего (около 9% задолженностей) имеют задолженности по кредиту клиенты с категорией дохода E (доход менее 30000), а реже всего - клиенты с категорией дохода D (доход более 30000 и менее 50000, 6% задолженностей) и клиенты с категорией дохода B (доход более 200000 и менее 1000000, 7% задолженностей). Клиенты с категорией дохода A (более 1000000) и категорией дохода C (более 50000 и менее 200000) имеют показатели 8% и 8.5% задолженностей соответственно.

##### Вопрос 4:

Как разные цели кредита влияют на его возврат в срок?

##### Вывод 4:

Составим сводную таблицу с группировкой по категории цели кредита.

In [31]:
#создание сводной таблицы с группировкой по категории цели кредита
#и со средним значением по столбцу debt
data.pivot_table(index='purpose_category', values='debt', aggfunc='mean')

,debt
purpose_category,
операции с автомобилем,0.093590
операции с недвижимостью,0.072334
получение образования,0.092200
проведение свадьбы,0.080034


Из полученной сводной таблицы можно сделать вывод, что клиенты чаще всего возвращают вовремя кредиты, взятые на операции с недвижимостью (около 7.2% задолженностей) и на проведение свадьбы (около 8% задолженностей), а реже всего - кредиты, взятые на операции с автомобилем (около 9.4% задолженностей) и на получение образования (около 9.2% задолженностей). 

## Общий вывод:

В ходе данного проекта были изучены данные кредитного отдела банка и проведена их предобработка. Были выявлены и заполнены пропуски в столбцах `days_employed` и `total_income`, выявлены аномалии в столбце `days_employed`, изменен тип данных в столбце `total_income`, а также были удалены строки-дубликаты. Для оптимизации работы с данными были созданы 2 датафрейма-"словаря", один с уровнем образования и идентификатором уровня образования, а другой с семейным положением и идентификатором семейного положения. Также значения уровня дохода и цели кредита были объединены по категориям. 

Было исследовано влияние различных факторов на возврат кредита в срок и получены следующие выводы:
1) Чаще всего кредиты в срок возвращают клиенты без детей, а многодетные клиенты чаще всего имеют задолженности по кредитам;

2) Вдовцы и вдовы реже всего имеют задолженности по кредитам, а неженатые/незамужние клиенты и клиенты в гражданском браке чаще имеют проблемы с возвратом кредита вовремя;

3) Клиенты с доходом менее 30000 чаще имеют задолженности по кредитам, а реже всего - клиенты с доходом от 30000 до 50000 и клиенты с доходом от 200000 до 1000000;

4) Кредиты, взятые на операции с автомобилем и на получение образования, чаще всего не возвращаются вовремя, а кредиты, взятые на операции с недвижимостью и проведение свадьбы, клиенты чаще возвращают в срок.

Также для всех факторов показатели задолженности не сильно отличаются друг от друга и не превышают 10.5%.